In [1]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.1:8b",
    temperature=0.9,
    keep_alive="20m"
)

In [2]:
def read_prompt(prompt_path : str) -> str:
    with open(prompt_path, 'r', encoding='utf-8') as file:
        prompt = file.read()
    return prompt

In [5]:

from langchain_core.messages import AIMessage
from langchain_core.prompts import ChatPromptTemplate

prompt_text = read_prompt("prompt/gen_answer.md")
prompt_template = ChatPromptTemplate.from_messages(["system", prompt_text])

gen_answer_chain = prompt_template | llm

In [6]:
import pandas as pd

data = pd.read_csv("question_gen_tv360.csv")

In [7]:
from tqdm import tqdm

# Add tqdm for progress tracking
for index, row in tqdm(data.iterrows(), total=len(data), desc="Processing films"):
    film_info = row["film_info"]
    
    # Generate answers
    ans_actor = gen_answer_chain.invoke({"film_information": film_info, "user_query": row["q_actor"]})
    ans_director = gen_answer_chain.invoke({"film_information": film_info, "user_query": row["q_director"]})
    ans_categories = gen_answer_chain.invoke({"film_information": film_info, "user_query": row["q_genres"]})
    ans_description = gen_answer_chain.invoke({"film_information": film_info, "user_query": row["q_plot"]})
    ans_countries = gen_answer_chain.invoke({"film_information": film_info, "user_query": row["q_country"]})

    # Store the results in the DataFrame
    data.at[index, "ans_actor"] = ans_actor.content
    data.at[index, "ans_director"] = ans_director.content
    data.at[index, "ans_categories"] = ans_categories.content
    data.at[index, "ans_description"] = ans_description.content
    data.at[index, "ans_countries"] = ans_countries.content

Processing films:   0%|          | 0/168 [00:00<?, ?it/s]

Processing films: 100%|██████████| 168/168 [17:10<00:00,  6.14s/it]


In [8]:
data.to_csv("question_answer_gen_tv360.csv", index=False)